In [2]:
import os
from datetime import datetime
from io import BytesIO
from os.path import isfile
from time import sleep

import numpy as np
import pandas as pd
import polars as pl
import psycopg2
import requests
from dotenv import load_dotenv
from PIL import Image
from telegram import Bot
from tqdm.notebook import tqdm

In [3]:
load_dotenv()
DATABASE_URL = os.environ['DATABASE_URL']
conn = psycopg2.connect(DATABASE_URL)

In [4]:
user_df = pl.read_database('select * from public.user', conn)
meme_source_df = pl.read_database('select id, type, url, status, language_code, parsed_at, created_at, updated_at, added_by from public.meme_source', conn)

In [5]:
meme_df = pl.read_database("select id, meme_source_id, raw_meme_id, status, type, telegram_file_id, caption, language_code, created_at, updated_at, published_at, ocr_result #>> '{}' ocr_result from public.meme", conn)

In [6]:
conn.close()

In [7]:
general_ru = [
    'https://vk.com/mysterious_conditions',
    'https://vk.com/wtf.rasha',
    'https://vk.com/demotiva',
    'https://vk.com/papkapic',
]

In [8]:
test_memes = (
    meme_df
    .join(meme_source_df.filter(pl.col('url').is_in(general_ru)), left_on='meme_source_id', right_on='id')
    .filter(pl.col('status') == 'ok')
    .sort('created_at', descending=True)
)

In [9]:
bot = Bot(os.environ['TELEGRAM_TOKEN'])

In [10]:
for row in tqdm(test_memes.iter_rows(named=True)):
    meme_id = row['id']
    fn = f'images/general_ru/{meme_id}.png'
    if isfile(fn):
        continue
    try:
        telegram_file = await bot.get_file(row['telegram_file_id'], read_timeout=30)
    except:
        print(row)
    r = requests.get(telegram_file.file_path)
    image_data = r.content
    img = Image.open(BytesIO(image_data))
    img.save(fn)

0it [00:00, ?it/s]

{'id': 8992724, 'meme_source_id': 74, 'raw_meme_id': 1631628, 'status': 'ok', 'type': 'image', 'telegram_file_id': 'AgACAgIAAx0EdEW6nAABAYGnZkqJr-6BbIKdGwS1_lPfCeb65SUAAj7mMRurXlFK3YBQk6ld0OABAAMCAAN4AAM1BA', 'caption': '', 'language_code': 'ru', 'created_at': datetime.datetime(2024, 5, 19, 23, 21, 11, 289079), 'updated_at': datetime.datetime(2024, 6, 9, 19, 32, 31, 992108), 'published_at': datetime.datetime(2024, 5, 19, 20, 0), 'ocr_result': '{"text": "убиваю время что демаешь ax так тогда я тоже буду тебя убивать cyanide and happiness explosm net", "model": "mystic:uriel/easyocr-r:v31", "raw_result": {"id": "run_2b51f55e440d4a32a1efbdfeb82ba215", "error": null, "state": "completed", "inputs": null, "outputs": [{"file": null, "type": "array", "value": [[[[425, 47], [649, 47], [649, 83], [425, 83]], "УБИВАЮ ВРЕМЯ ."], [[[51, 79], [263, 79], [263, 119], [51, 119]], "ЧТО ДЕМАЕШЬ?"], [[[31, 408], [333, 408], [333, 481], [31, 481]], "AX ТАК! ТОГДА Я ТОЖЕ БУДУ ТЕБЯ УБИВАТЬ!"], [[[243, 736],